In [29]:
import layer
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split     


data=pd.read_csv('birds.csv')
gender_distribution = data.groupby('bird category')['gender'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
data['gender'] = data.apply(lambda row: gender_distribution[row['bird category']] if pd.isnull(row['gender']) else row['gender'], axis=1) 
label_encoder=preprocessing.LabelEncoder()
data.iloc[:,0]=label_encoder.fit_transform(data.iloc[:,0])


hot_encoder = preprocessing.OneHotEncoder(sparse_output=False) 
encoded_columns = hot_encoder.fit_transform(data.iloc[:,-1].values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded_columns, columns=hot_encoder.get_feature_names_out([data.columns[-1]]))


# Drop the original last column and append one-hot-encoded columns
data = pd.concat([data.drop(data.columns[-1], axis=1), encoded_df], axis=1)

normalizer=preprocessing.MinMaxScaler()
data.iloc[:,1:5]=normalizer.fit_transform(data.iloc[:,1:5]).astype(float)
X=data.iloc[:,0:5]
Y=data.iloc[:,5:]

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.4,stratify=Y,shuffle=True)
X.shape

C:\Users\20155\AppData\Local\Temp\ipykernel_10656\1321522823.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.29166667 0.30555556 0.15277778 0.22222222 0.20833333 0.26388889
 0.25694444 0.54861111 0.21527778 0.43055556 0.16666667 0.27777778
 0.13888889 0.30555556 0.47222222 0.27777778 0.20833333 0.5
 0.17361111 0.41666667 0.19444444 0.25       0.30555556 0.34722222
 0.30555556 0.30555556 0.23611111 0.13888889 0.125      0.34722222
 0.15277778 0.33333333 0.16666667 0.33333333 0.17361111 0.40277778
 0.34722222 0.23611111 0.16666667 0.54166667 0.125      0.33333333
 0.11111111 0.47222222 0.08333333 0.52777778 0.20138889 0.07638889
 0.20833333 0.40277778 0.5        0.83333333 0.48611111 0.83333333
 0.75       0.51388889 0.58333333 0.69444444 0.47222222 0.68055556
 0.54166667 0.79166667 0.54166667 0.875      0.41666667 0.875
 0.40277778 1.         0.58333333 0.73611111 0.83333333 0.63888889
 0.47222222 0.65277778

(150, 5)

- we need to create layer 
- we need in all layer n neurans 

#### layer 

layer is list have n neurans work seperated 

so we need to create single neuran fist then make layer

To make matrix for W (weight) it will by n*m
- where n is number of features
- m is number of neurans

<p>[0.21 , -0.4]<br>[0.15 , 0.1]<p> previous matrix is W 

- where first col is w1
- second col is w2

In [2]:
# first layer 
# W = np.random.rand(data.shape , units)
# bias = np.random.rand(units)
X =np.array([0,0])
W = np.array([[0.21,-0.4],
              [0.15,0.1]])
bias = np.array([-0.3,0.25])


In [3]:
def sigmoid(Z):
    return round((1/(1+math.exp(-Z))) , 3)

In [4]:
#  a_out = np.zeros(units)
#  for j in range(units):
#   Z = np.dot(X , W[: , j]) + bias[j]
#   a_out[j] = activation(function_name , Z)

Z_1 = np.dot(X , W[: , 0]) + bias[0]
Z_2 = np.dot(X , W[: , 1])+bias[1]
print(Z_1)
print(Z_2)

-0.3
0.25


In [5]:
welcome = "WelCome"
print(welcome.lower())

welcome


In [92]:
class hideen_Layer:
    def __init__(self , neurons , activation_function , data):
        self.neurons = int(neurons)
        self.activation_function =activation_function
        self.data = data
        # W = np.random.rand(data.shape[1] ,  self.neurons)
        self.W = np.random.rand(self.data.shape[0] , self.neurons)
        self.bias = np.random.rand(self.neurons)
        self.a_out= np.zeros(self.neurons) # output from each neuron (unit)
        self.error = np.zeros(self.neurons)
        self.differentiating = np.zeros(self.neurons)
        self.a = 1
        self.b = 1
        
        self.train()
    # activation function & their differentiating
    #%% 
    def sigmoid(self,Z , a =1):
        self.a = a
        return (1/(1+math.exp(-a*Z)))
    
    def sigmoid_differentiating(self,Z , a = 1):
        self.a = a
        y =self.sigmoid(Z,  self.a)
        
        return self.a*y*(1-y)
    
    # did you need to store a and b ??
    
    def hyperbolic_tangent(self,Z , b =1):
        self.b = b
        top = 1-math.exp(- self.b*Z)
        down = 1 + math.exp(- self.b*Z)   
        return top/down
    
    def hyperbolic_tangent_differentiating(self,Z , a =1 , b = 1):
        outer = b/a
        y = self.hyperbolic_tangent(Z , b)
        return outer*(a-y)*(a+y)
    #%%
    # select activation function
    def activation(self , Z ,index,  a = 1):
        if self.activation_function.lower() =="sigmoid":
            self.differentiating[index] = self.sigmoid_differentiating(Z)
            return  self.sigmoid(Z , a)
        elif self.activation_function.lower() =="hyperbolic_tangent" :
            self.differentiating[index] = self.hyperbolic_tangent_differentiating(Z)
            return self.hyperbolic_tangent(Z , a)
        else:
            return Exception
    # generate output
    def train(self):
        for j in range(self.neurons):
              Z = np.dot(self.data , self.W[: , j])+self.bias[j]
              try :
                  self.a_out[j] =self.activation(Z , j)
              except :
                  self.a_out= np.zeros(self.neurons)
                  print( "Error in selected function")
                  return 
    def backpropagate(self):
        pass 
    def forword_propagate_after_back(self):
        pass
        
    

In [7]:
first_layer = hideen_Layer(2 , "sigmoid" , X)

In [8]:
first_layer.train()

In [9]:
first_layer.a_out

array([0.51661935, 0.70878298])

In [10]:
second_layer = hideen_Layer(2 , "sigmoid" , X)
second_layer.train()
second_layer.a_out

array([0.58837525, 0.66808169])

- storge data of each layer in list
- we have layer so we need number of layers 

In [ ]:
class sequence :
    def __init__(self , num_of_layers ,epoch  , data):
        self.layers = []
        self.num_of_layers = num_of_layers
        self.data  =data
        self.epoch = epoch
    def build_layers(self):
        
        # form GUI take units and activation
        
        self.layers.append(hideen_Layer(num_of_units , activation , self.data))
        for i in range(self.num_of_layer-1):
            previous_layer = self.layers[i]
            self.layers.append(hideen_Layer(num_of_units , activation , previous_layer.a_out))
        
        previous_layer = self.layers[-1]
        self.layers.append(hideen_Layer(3 , activation , previous_layer.a_out))
        
    def back_propagation(self):
        len_of_layers = len(self.layers)-1
        start_index = len_of_layers - 1

        for i in range(len_of_layers):
            for row in range(self.layers[start_index-i].neurons):
                error_term=np.dot(self.layers[start_index-i+1].error , self.layers[start_index-i+1].W[row , :])
                self.layers[start_index-i].error[row] =self.layers[start_index-i].differentiating[row]*error_term
                
    # decide where learning rate will put
    def forward_propagation(self):
        for layer in self.layers:
            for neuron in range(layer.neurons):
                layer.W[: , neuron] = layer.W[: , neuron] + learning_rate * layer.data * layer.error[neuron]
                layer.bias[neuron] =  layer.bias[neuron] + learning_rate * layer.error[neuron]
                


In [99]:
# put input
layers=[]
num_of_layer = int(input("enter number of hidden layer"))
num_of_units = int(input("enter number of unints for unit 1"))
activation = input("enter activation function")
layers.append(hideen_Layer(num_of_units , activation , X))

for i in range(num_of_layer-1):
    num_of_units = int(input(f"enter number of unints for unit {i+2}"))
    activation = input("enter activation function")
    previous_layer = layers[i]
    layers.append(hideen_Layer(num_of_units , activation , previous_layer.a_out))
    
previous_layer = layers[-1]
layers.append(hideen_Layer(1 , "sigmoid" , previous_layer.a_out))


In [ ]:
for layer in layers :
    for neuron in range(layer.neurons):
        layer.W[: , neuron] = layer.W[: , neuron] + 0.5 * 

In [106]:
for layer in layers :
    for neuron in range(layer.neurons):
        print(layer.W[: , neuron])
    print("done")

[0.32099001 0.21438743]
[0.0498321  0.46760353]
[0.56389822 0.70890522]
[0.65312354 0.76950449]
[0.36953099 0.64446893]
done
[0.02306508 0.92149792 0.05851145 0.03916407 0.42374823]
[0.9685795  0.88186476 0.2765589  0.83877687 0.76463231]
[0.47913084 0.2407692  0.29234205 0.24835575 0.3627783 ]
[0.77305737 0.11990183 0.18884328 0.98809577 0.55524365]
done
[0.76860491 0.93967457 0.78471611 0.38553374]
done


In [107]:
print(len(layers))
for i in range(len(layers)):
    print(layers[i].W)

3
[[0.32099001 0.0498321  0.56389822 0.65312354 0.36953099]
 [0.21438743 0.46760353 0.70890522 0.76950449 0.64446893]]
[[0.02306508 0.9685795  0.47913084 0.77305737]
 [0.92149792 0.88186476 0.2407692  0.11990183]
 [0.05851145 0.2765589  0.29234205 0.18884328]
 [0.03916407 0.83877687 0.24835575 0.98809577]
 [0.42374823 0.76463231 0.3627783  0.55524365]]
[[0.76860491]
 [0.93967457]
 [0.78471611]
 [0.38553374]]


In [93]:
for i in range(len(layers)):
    print(layers[i].data)

[0 0]
[0.64318728 0.53353054 0.56620692 0.69683459]
[0.79079906 0.86500161]
[0.72950588 0.7789295  0.67424649]


In [95]:
for i in range(len(layers)):
    print(layers[i].W)
    print()

[[0.52389206 0.86620289 0.96367209 0.41114226]
 [0.39491797 0.15781282 0.72905456 0.22407595]]

[[0.36149756 0.16380307]
 [0.7189759  0.85637456]
 [0.36072133 0.79438506]
 [0.17578366 0.00361792]]

[[0.51366753 0.75617735 0.29173687]
 [0.55162012 0.07164045 0.18493112]]

[[0.16311287]
 [0.44194065]
 [0.81963896]]



In [96]:
# print(len(layers))
for i in range(len(layers)):
    print(layers[i].error)

[6.89050921e-05 1.78849194e-04 1.09066721e-04 2.71653824e-05]
[0.00072681 0.00022895]
[0.00111696 0.00264091 0.00624729]
[0.03470254]


In [ ]:
layers[0].W[: , 0] # W1
layers[0].W[0 , :] # W1[0] , W2[0] , ... Wn[0] where n in is num of neurons

array([0.52389206, 0.86620289, 0.96367209, 0.41114226])

In [41]:
print(layers[0].W[0,:] * [1,2,3,4])

[0.52389206 1.73240578 2.89101626 1.64456906]


In [ ]:
target = np.zeros(layers[-1].neurons)
target[0] = 1
print(target)

In [15]:
def sigmoid(Z , a =1):
    return (1/(1+math.exp(-a*Z)))

def sigmoid_differentiating(y , a = 1):
    return a*y*(1-y)


In [74]:
layers[-1].error=(target-layers[-1].a_out)*sigmoid_differentiating( layers[-1].a_out)
print(layers[-1].error)

[0.03470254]


In [70]:
looping = len(layers)-1
looping2 = looping-1
for i in range(looping):
    print(looping2-i)

2
1
0


In [72]:
len_of_layers = len(layers)-1
start_index = len_of_layers - 1

for i in range(len_of_layers):
        for row in range(layers[start_index-i].neurons):
                error_term=np.dot(layers[start_index-i+1].error , layers[start_index-i+1].W[row , :])
                layers[start_index-i].error[row] = sigmoid_differentiating(  layers[start_index-i].a_out[row])*error_term

In [86]:
for i in range(len(layers)):
    print(layers[i].a_out)
    print()

[0.64318728 0.53353054 0.56620692 0.69683459]

[0.79079906 0.86500161]

[0.72950588 0.7789295  0.67424649]

[0.79047462]



In [42]:
for i in range(len(layers)):
    print(layers[i].W)
    print()

[[0.52389206 0.86620289 0.96367209 0.41114226]
 [0.39491797 0.15781282 0.72905456 0.22407595]]

[[0.36149756 0.16380307]
 [0.7189759  0.85637456]
 [0.36072133 0.79438506]
 [0.17578366 0.00361792]]

[[0.51366753 0.75617735 0.29173687]
 [0.55162012 0.07164045 0.18493112]]

[[0.16311287]
 [0.44194065]
 [0.81963896]]



In [91]:
error_term = np.dot(layers[2].W[1 , :],layers[2].error)
error_for_second = error_term * sigmoid_differentiating(layers[1].a_out[1])
print(error_for_second)


0.00022895257372403363


In [73]:
for i in range(len(layers)):
    print(layers[i].error)


[6.89050921e-05 1.78849194e-04 1.09066721e-04 2.71653824e-05]
[0.00072681 0.00022895]
[0.00111696 0.00264091 0.00624729]
[0.03470254]


In [30]:
print(len(layers))
print(layers[0].a_out)
print(layers[1].a_out)
print(layers[2].a_out)
print(layers[3].a_out)

4
[0.64318728 0.53353054 0.56620692 0.69683459]
[0.79079906 0.86500161]
[0.72950588 0.7789295  0.67424649]
[0.79047462]


In [16]:
def calculate_error_output(y , target):
    return (target[0] - y)*sigmoid_differentiating(y)

In [75]:
def train(W , bias , X):
    units = W.shape[1] # shape 1 is number of col that indicate of neurans
    a_out = np.zeros(units)
    
    for j in range(units):
        Z = np.dot(W[:,j],X)+bias[j]
        a_out[j] = sigmoid(Z)
    return a_out

In [76]:
# first layer
W = np.array([[0.21 , -0.4],
              [0.15,0.1]])
bias = np.array([-0.3,0.25])

a_out= train(W,bias ,X)
print(a_out)


[0.42555748 0.5621765 ]


In [77]:
len(layers)

4

In [78]:
W_output = np.array([[-0.2],
                    [0.3]])
bias_out = np.array([-0.4])
output_layer =train(W_output,bias_out ,a_out)
print(output_layer)

[0.42153907]


In [85]:
storge_data= [[W,bias,a_out], [W_output ,bias_out, output_layer]]
storge_data

[[array([[ 0.21, -0.4 ],
         [ 0.15,  0.1 ]]),
  array([-0.3 ,  0.25]),
  array([0.42555748, 0.5621765 ])],
 [array([[-0.2],
         [ 0.3]]),
  array([-0.4]),
  array([0.42153907])]]

In [80]:
target = np.array([0])
error= []
error


[]

In [81]:
def train_for_output(W , bias , X):
    units = W.shape[1] # shape 1 is number of col that indicate of neurans
    a_out = np.zeros(units)
    
    for j in range(units):
        Z = np.dot(W[:,j],X)+bias[j]
        a_out[j] = Z
    return a_out

In [82]:
# y = np.dot(W_output , a_out) + bias_out
print(calculate_error_output( output_layer, target))

[-0.10278972]


In [83]:
# net is W * X + bias
# error for putput (target - output)*differentiating(net)
# error in each unit = differentiating(net) *sum(error in previous *their W)

In [84]:
def calculate_error_output(y , target):
    return (target[0] - y)*sigmoid_differentiating(y)